# Reconstruction Tests

In [1]:
%matplotlib widget

In [2]:
import time
import tomopy
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from matplotlib.patches import Rectangle, Circle, Polygon
from matplotlib.path import Path
import timeit

# Non-negative ART Reconstruction

In [3]:
def ct(ds,algorithm='art', stripe_removal=False):
    
    ds['X'] = (ds['X']-ds['X'][0])-((ds['X'][-1]-ds['X'][0])/2)
    
    proj = np.swapaxes(ds.values, 1, 2)

    if stripe_removal:
        proj = tomopy.prep.stripe.remove_all_stripe(proj_,snr=3,sm_size=21,la_size=64,dim=1,ncore=None,nchunk=None)

    print('running')
    recon = tomopy.recon(proj, np.deg2rad(ds.Phi.values), center=int(ds.shape[1]/2), algorithm=algorithm, sinogram_order=False)

    da = xr.DataArray(data=recon.astype('float32'), coords=[ds.Q,ds.X,ds.X],dims=['radial', 'vertical', 'horizontal'])

    #dia = ds.X[-1]-ds.X[0]
    
    #da = da.sel(vertical=slice(-dia.values/2,+dia.values/2)).sel(horizontal=slice(-dia.values/2,+dia.values/2))
    da['distances'] = np.sqrt((da.horizontal)**2 + (da.vertical)**2) 
    
    da = da.where(da >= 0, 0)

    return(da)

# CT Recon Image Plot (originally from Mehmet Topsakal)

In [4]:
def xrdct_plot(
        da_recon,
        radial_range=(1,5.5), circular_mask = True,
        ax=None, 
        robust=False, vmin=None, vmax=None, cmap='Greys',
        ax_ticks=True, cbar = True
        ):
    
    da_recon_sel = da_recon.sel(radial=slice(radial_range[0],radial_range[1]))

    if circular_mask:
        obj_radius = (da_recon_sel['vertical'][-1]-da_recon_sel['vertical'][0])/2
        da_recon_sel = da_recon_sel.where(da_recon_sel.distances<np.sqrt(1)*obj_radius).sel(horizontal=slice(-obj_radius,obj_radius)).sel(vertical=slice(-obj_radius,obj_radius))
    

    if cbar: 
        if ax_ticks:
            cbar_kwargs=dict(orientation="horizontal", shrink=0.6)
            add_colorbar = True
        else:
            cbar_kwargs=dict(orientation="horizontal", shrink=0.6)
            add_colorbar = True
    else:
        cbar_kwargs=None
        add_colorbar = False

    da_recon_sel.mean(dim='radial').plot.imshow(ax=ax,
                                        robust=robust,
                                        yincrease=True,
                                        add_colorbar=add_colorbar,
                                        cbar_kwargs=cbar_kwargs,
                                        cmap=cmap,
                                        vmin=vmin,
                                        vmax=vmax
                                            )

    if ax_ticks:
        ax.set_aspect('equal')
        ax.set_xlabel('Horizontal (mm)')
        ax.set_ylabel('Vertical (mm)')
    else:
        ax.set_aspect('equal')
        ax.set_xlabel(None)
        ax.set_ylabel(None)
        ax.set_xticks([])
        ax.set_yticks([])        
        
    return

Define a circle on a reconstruction image to select data

In [5]:
def circle_select(recon, center_x, center_y, radius):

    x = recon.coords['horizontal'].values
    y = recon.coords['vertical'].values

    distances = np.sqrt((x[:, None] - center_x)**2 + (y[None, :] - center_y)**2)

    circle_mask = distances < radius

    return(recon.where(circle_mask))

Define a polygon on a reconstruction image to select data

In [6]:
def polygon_select(recon, vertices):
    #recon is the reconstruction dataarray
    #vertices is a list of tuples defining the vertices of the polygon
    path = Path(np.array(vertices))
    
    x, y = np.meshgrid(recon.coords['horizontal'].values, recon.coords['vertical'].values)

    polygon_mask = path.contains_points(np.stack((x.ravel(),y.ravel()),axis=1)).reshape(recon.distances.shape)

    return(recon.where(polygon_mask))

# Convert Data Array to .qchi File

In [8]:
def DA_to_qchi(DA, string, x):
    if x==True:
        df = pd.DataFrame(
            {
                'Q' : DA.radial.values,
                'Intensities' : DA.values
                }
            )
    else:
        df = pd.DataFrame(
            {
                'Q' : DA.Q.values,
                'Intensities' : DA.values
                }
            )
        
    
    df.to_csv(string+'.qchi',sep=' ',index=False)

    return()

# Test a series of pixels centers for reconstruction

In [10]:
def ct_center(ds):
    ds['X'] = (ds['X']-ds['X'][0])-((ds['X'][-1]-ds['X'][0])/2)
    proj = np.swapaxes(ds.values, 1, 2)
    recon = tomopy.write_center(proj, 
                               np.deg2rad(ds.Phi.values), 
                               cen_range=(int(ds.shape[1]/2)-5,int(ds.shape[1]/2)+5,10), 
                               algorithm='gridrec', 
                               sinogram_order=False)
    return()

In [11]:
def ct_ads(ds,cen, algorithm='art', stripe_removal=False):
    
    ds['X'] = (ds['X']-ds['X'][0])-((ds['X'][-1]-ds['X'][0])/2)
    
    proj = np.swapaxes(ds.values, 1, 2)

    if stripe_removal:
        proj = tomopy.prep.stripe.remove_all_stripe(proj_,snr=3,sm_size=21,la_size=64,dim=1,ncore=None,nchunk=None)

    print('running')
    recon = tomopy.recon(proj, np.deg2rad(ds.Phi.values), center=cen, algorithm=algorithm, sinogram_order=False)

    da = xr.DataArray(data=recon.astype('float32'), coords=[ds.Q,ds.X,ds.X],dims=['radial', 'vertical', 'horizontal'])

    da['distances'] = np.sqrt((da.horizontal)**2 + (da.vertical)**2) 
    
    da = da.where(da >= 0, 0)

    return(da)

In [ ]:
mono_list = []
cen_list = [41,42,43,44,45,46,47,48,49,50,51,51]
for i in cen_list:
    globals()["a_monolith_"+str(i)] = ct_ads(ads['XRD_CT'],cen=i)
    mono_list.append(globals()["a_monolith_"+str(i)])
print(mono_list)

running
running
running
running
running
running
running
running
running
running
running
running
[<xarray.DataArray (radial: 5800, vertical: 94, horizontal: 94)> Size: 205MB
array([[[233.7181    , 106.09914   ,  75.584656  , ...,  31.193102  ,
          42.060444  , 201.78838   ],
        [  0.        ,   9.285305  ,  18.894527  , ...,  24.060305  ,
          15.801062  ,  63.709866  ],
        [  5.233297  ,   9.9580965 ,  12.340849  , ...,  21.958374  ,
          11.895238  ,  41.294518  ],
        ...,
        [  0.        ,   0.        ,   0.        , ...,   0.71431416,
           0.92661196,   6.5726705 ],
        [  0.        ,   0.        ,   0.        , ...,   0.626741  ,
           0.7615512 ,   5.842932  ],
        [  0.        ,   0.        ,   0.        , ...,   1.0040394 ,
           0.6826183 ,   4.1272097 ]],

       [[232.6772    , 105.799675  ,  75.3495    , ...,  31.144196  ,
          42.393127  , 201.09772   ],
        [  0.        ,   9.1929865 ,  18.672874  , ..., 

# Output a series of pixels as .chi files

In [373]:
composite = ct_ads(ds_composite['XRD_CT'],cen=44)
composite.to_netcdf('CeUiO66-SiO2_PDFCT_ART.nc')
composite

running


<xarray.DataArray (radial: 5800, vertical: 88, horizontal: 88)> Size: 180MB
array([[[0.        , 0.        , 0.        , ..., 1.2174568 ,
         0.88360107, 0.        ],
        [1.3671522 , 1.1987996 , 1.4267573 , ..., 0.7660391 ,
         0.6192853 , 0.        ],
        [3.5918643 , 3.1685433 , 3.6775386 , ..., 0.95790607,
         0.8022126 , 0.        ],
        ...,
        [0.        , 0.3544926 , 0.55883247, ..., 3.3381696 ,
         3.0408735 , 3.1095064 ],
        [0.        , 0.16450405, 0.661122  , ..., 1.3150792 ,
         0.5285376 , 1.1629316 ],
        [0.        , 0.9052164 , 0.92559004, ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.        , ..., 1.3018948 ,
         0.8760514 , 0.        ],
        [1.2850446 , 1.1783254 , 1.4516929 , ..., 0.6671116 ,
         0.6367294 , 0.        ],
        [3.5496583 , 3.1705985 , 3.591165  , ..., 0.99809587,
         0.7139416 , 0.        ],
...
        [0.        , 0.7689671 , 1.0035242 , ..., 2.95297   ,
         2.5862064 , 2.592183  ],
        [0.        , 0.7085251 , 1.081817  , ..., 1.4161954 ,
         0.9842311 , 1.1072736 ],
        [0.        , 1.2859399 , 1.7137724 , ..., 0.03277849,
         0.        , 0.        ]],

       [[0.        , 0.        , 0.01620083, ..., 2.0596373 ,
         1.3943498 , 0.        ],
        [1.1637937 , 1.0485584 , 1.4422691 , ..., 1.3553001 ,
         0.5905619 , 0.        ],
        [2.6518445 , 2.5060503 , 3.0462878 , ..., 1.2807804 ,
         0.7067472 , 0.        ],
        ...,
        [0.        , 0.7751226 , 0.9557121 , ..., 3.0247426 ,
         2.6335773 , 2.6690452 ],
        [0.        , 0.63918835, 1.1065917 , ..., 1.4338775 ,
         0.9697353 , 1.0429682 ],
        [0.        , 1.3085102 , 1.7167908 , ..., 0.2144706 ,
         0.        , 0.        ]]], dtype=float32)
Coordinates:
  * radial      (radial) float64 46kB 0.5025 0.5075 0.5125 ... 29.49 29.49 29.5
  * vertical    (vertical) float64 704B -0.66 -0.6448 -0.6297 ... 0.6448 0.66
  * horizontal  (horizontal) float64 704B -0.66 -0.6448 -0.6297 ... 0.6448 0.66
    distances   (horizontal, vertical) float64 62kB 0.9334 0.9227 ... 0.9334

In [292]:
path_to_composite = '/Users/danielonolan/Library/CloudStorage/GoogleDrive-danolimerick07@gmail.com/My Drive/XPD_Nov2024/MOF-SiO2/Composite_to_PDF_then_NMF/'
for i in composite.horizontal.values:
    for j in composite.vertical.values:
        composite_pixel = composite.sel(horizontal=i,method='nearest').sel(vertical=j,method='nearest')
        DA_to_qchi(
            composite_pixel,
            path_to_composite+'MOF-SiO2_ART_Horizontal='+str(i)+'_Vertical='+str(j),
            True
        )

# Non-negative Least Squares demo

In [658]:
composite_all = quartz_id.to_numpy()
composite_all = np.reshape(composite_all,(5800,7744)).transpose()
composite_all = np.nan_to_num(composite_all, nan=0)

In [635]:
from scipy.optimize import minimize
def nnls_alternative(A, b):
    # Objective function to minimize: ||Ax - b||^2
    def objective(x):
        return(np.linalg.norm(A @ x - b)**2)

    # Non-negativity constraint
    bounds = [(0, None) for _ in range(A.shape[1])]

    # Initial guess
    x0 = np.zeros(A.shape[1])

    # Optimization
    result = minimize(objective, x0, bounds=bounds)

    return(result.x)

x = nnls_alternative(np.transpose(composite_all), composite.sel(horizontal=-0.087,method='nearest').sel(vertical=0.266,method='nearest').values)
print(x)

[0. 0. 0. ... 0. 0. 0.]


In [637]:
x = x.reshape((88,88))

In [654]:
Composite_SiO2_NNLS = xr.DataArray(x, coords=[composite.vertical, composite.horizontal], dims=('vertical','horizontal'))
Composite_SiO2_NNLS['distances'] = np.sqrt((Composite_SiO2_NNLS.horizontal)**2 + (Composite_SiO2_NNLS.vertical)**2) 
Composite_SiO2_NNLS

<xarray.DataArray (vertical: 88, horizontal: 88)> Size: 62kB
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * vertical    (vertical) float64 704B -0.66 -0.6448 -0.6297 ... 0.6448 0.66
  * horizontal  (horizontal) float64 704B -0.66 -0.6448 -0.6297 ... 0.6448 0.66
    distances   (horizontal, vertical) float64 62kB 0.9334 0.9227 ... 0.9334

# Non-negative Matrix Factorization using .nc CT data

In [190]:
from sklearn.decomposition import NMF
test = composite.to_numpy()
test = np.reshape(test,(5800,7744))

model = NMF(n_components=3, init='random', random_state=0, max_iter=10000)
W = model.fit_transform(test)
H = model.components_

data_reconstructed = np.dot(W, H)
data_reconstructed = np.reshape(data_reconstructed, (5800,88,88))

reconstructed_data_array = xr.DataArray(data_reconstructed, coords=composite.coords, dims=composite.dims)
reconstructed_data_array

<xarray.DataArray (radial: 5800, vertical: 88, horizontal: 88)> Size: 180MB
array([[[0.0000000e+00, 0.0000000e+00, 9.5866487e-04, ...,
         1.6099210e+00, 9.8559636e-01, 0.0000000e+00],
        [1.2852842e+00, 1.0289453e+00, 1.3798585e+00, ...,
         8.0930161e-01, 4.1838092e-01, 0.0000000e+00],
        [3.3652418e+00, 2.9392347e+00, 3.4258981e+00, ...,
         7.2551572e-01, 6.0094827e-01, 0.0000000e+00],
        ...,
        [0.0000000e+00, 4.6738684e-01, 5.6662369e-01, ...,
         3.5414608e+00, 2.8369741e+00, 3.2759058e+00],
        [0.0000000e+00, 2.6561701e-01, 5.9087080e-01, ...,
         1.1634958e+00, 7.6036882e-01, 1.2414173e+00],
        [0.0000000e+00, 8.3460122e-01, 1.3919575e+00, ...,
         9.6224278e-04, 0.0000000e+00, 0.0000000e+00]],

       [[0.0000000e+00, 0.0000000e+00, 5.3057208e-04, ...,
         1.5994933e+00, 9.7888976e-01, 0.0000000e+00],
        [1.2712241e+00, 1.0235577e+00, 1.3697356e+00, ...,
         8.0302435e-01, 4.1553399e-01, 0.0000000e+00],
        [3.3458641e+00, 2.9287219e+00, 3.4065831e+00, ...,
         7.2057891e-01, 5.9965438e-01, 0.0000000e+00],
...
        [0.0000000e+00, 4.4217730e-01, 5.3606158e-01, ...,
         2.8516407e+00, 2.2991982e+00, 2.6383424e+00],
        [0.0000000e+00, 2.5129038e-01, 5.5900085e-01, ...,
         1.1141222e+00, 6.0647333e-01, 9.9498135e-01],
        [0.0000000e+00, 7.8958517e-01, 1.3168793e+00, ...,
         9.1034209e-04, 0.0000000e+00, 0.0000000e+00]],

       [[0.0000000e+00, 0.0000000e+00, 3.8430633e-03, ...,
         1.4945667e+00, 9.3206489e-01, 0.0000000e+00],
        [1.0516697e+00, 7.3850602e-01, 1.2146856e+00, ...,
         7.6824379e-01, 3.9565706e-01, 0.0000000e+00],
        [2.6481204e+00, 2.3073523e+00, 2.8575590e+00, ...,
         6.8611026e-01, 4.6600735e-01, 0.0000000e+00],
        ...,
        [0.0000000e+00, 4.4200131e-01, 5.3584820e-01, ...,
         2.8510191e+00, 2.2986805e+00, 2.6377392e+00],
        [0.0000000e+00, 2.5119036e-01, 5.5877835e-01, ...,
         1.1136584e+00, 6.0633278e-01, 9.9474132e-01],
        [0.0000000e+00, 7.8927088e-01, 1.3163551e+00, ...,
         9.0997975e-04, 0.0000000e+00, 0.0000000e+00]]], dtype=float32)
Coordinates:
  * radial      (radial) float64 46kB 0.5025 0.5075 0.5125 ... 29.49 29.49 29.5
  * vertical    (vertical) float64 704B -0.66 -0.6448 -0.6297 ... 0.6448 0.66
  * horizontal  (horizontal) float64 704B -0.66 -0.6448 -0.6297 ... 0.6448 0.66
    distances   (horizontal, vertical) float64 62kB 0.9334 0.9227 ... 0.9334

In [803]:
component = [1,2,3,4]

model_xmono = NMF(n_components=4, init='random', random_state=0, max_iter=10000)
W_xmono_c4_nmf = model_xmono.fit_transform(x_mono_nmf) #x_mono_nmf = flattend nc file
H_xmono_c4_nmf = model_xmono.components_

xmono_nmf_data_reconstructed = np.dot(W_xmono_c4_nmf, H_xmono_c4_nmf)
xmono_nmf_data_reconstructed = np.reshape(xmono_nmf_data_reconstructed, (5800,94,94))

reconstructed_xmono_nmf_data_array = xr.DataArray(xmono_nmf_data_reconstructed, coords=x_monolith.coords, dims=x_monolith.dims)

W_xmono_c4_nmf.shape
W_xmono_c4_nmf = xr.DataArray(W_xmono_c4_nmf, coords=[w_monolith.radial, component], dims=('radial','Component'))

H_xmono_c4_nmf = np.reshape(H_xmono_c4_nmf, (4,94,94))
H_xmono_c4_nmf = xr.DataArray(H_xmono_c4_nmf, coords=[component, x_monolith.vertical, x_monolith.horizontal], dims=('Component','vertical','horizontal'))
H_xmono_c4_nmf['distances'] = np.sqrt((H_xmono_c4_nmf.horizontal)**2 + (H_xmono_c4_nmf.vertical)**2) 

########################


model_wmono = NMF(n_components=4, init='random', random_state=0, max_iter=10000)
W_wmono_c4_nmf = model_wmono.fit_transform(w_mono_nmf) #W_mono_nmf = flattend nc file
H_wmono_c4_nmf = model_wmono.components_

wmono_nmf_data_reconstructed = np.dot(W_wmono_c4_nmf, H_wmono_c4_nmf)
wmono_nmf_data_reconstructed = np.reshape(wmono_nmf_data_reconstructed, (5800,94,94))

reconstructed_wmono_nmf_data_array = xr.DataArray(wmono_nmf_data_reconstructed, coords=w_monolith.coords, dims=w_monolith.dims)

W_wmono_c4_nmf.shape
W_wmono_c4_nmf = xr.DataArray(W_wmono_c4_nmf, coords=[w_monolith.radial, component], dims=('radial','Component'))

H_wmono_c4_nmf = np.reshape(H_wmono_c4_nmf, (4,94,94))
H_wmono_c4_nmf = xr.DataArray(H_wmono_c4_nmf, coords=[component, w_monolith.vertical, w_monolith.horizontal], dims=('Component','vertical','horizontal'))
H_wmono_c4_nmf['distances'] = np.sqrt((H_wmono_c4_nmf.horizontal)**2 + (H_wmono_c4_nmf.vertical)**2) 

########################